In [1]:
# First install required packages
!pip install transformers torch scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [30]:
import json
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import (
    CamembertTokenizer,
    CamembertForSequenceClassification,
    Trainer,
    TrainingArguments
)
import torch
from google.colab import files

In [31]:
# @title Step 2: Upload Your Dataset
uploaded = files.upload()
filename = next(iter(uploaded))
with open(filename, 'r', encoding='utf-8') as f:
    data = json.load(f)

Saving training_data.json to training_data.json


In [32]:
df = pd.DataFrame(data)
print(f"Loaded {len(df)} comments")
print("Category distribution:")
print(df['label'].value_counts())

Loaded 500 comments
Category distribution:
label
quality     138
delivery    127
price       124
service     111
Name: count, dtype: int64


In [33]:
# @title Step 3: Prepare Data
le = LabelEncoder()
df['encoded_label'] = le.fit_transform(df['label'])

In [34]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [35]:
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")


In [38]:
# Corrected tokenization function
def tokenize_text(text_list):
    return tokenizer(
        text_list,
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

In [39]:
# Tokenize the text data directly from the Series
train_encodings = tokenize_text(train_df['commentaire'].tolist())
val_encodings = tokenize_text(val_df['commentaire'].tolist())

In [40]:
class CommentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {
            key: val[idx] for key, val in self.encodings.items()
        }
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [41]:
train_dataset = CommentDataset(train_encodings, train_df['encoded_label'].values)
val_dataset = CommentDataset(val_encodings, val_df['encoded_label'].values)

In [42]:
# @title Step 4: Train Model
model = CamembertForSequenceClassification.from_pretrained(
    "camembert-base",
    num_labels=len(le.classes_)
)

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    report_to="none"
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [44]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)



In [45]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,0.761880
2,0.888700,0.371403
3,0.888700,0.239730
4,0.296100,0.209200


TrainOutput(global_step=100, training_loss=0.5923729801177978, metrics={'train_runtime': 2213.5715, 'train_samples_per_second': 0.723, 'train_steps_per_second': 0.045, 'total_flos': 105246312038400.0, 'train_loss': 0.5923729801177978, 'epoch': 4.0})

In [46]:
# @title Step 5: Save and Download Model
!zip -r model.zip ./results

model.save_pretrained("saved_model")
tokenizer.save_pretrained("saved_model")
pd.Series(le.classes_).to_csv("labels.csv", index=False)


updating: results/ (stored 0%)
updating: results/checkpoint-75/ (stored 0%)
updating: results/checkpoint-75/model.safetensors (deflated 15%)
updating: results/checkpoint-75/config.json (deflated 54%)
updating: results/checkpoint-75/rng_state.pth (deflated 24%)
updating: results/checkpoint-75/optimizer.pt (deflated 28%)
updating: results/checkpoint-75/scheduler.pt (deflated 56%)
updating: results/checkpoint-75/trainer_state.json (deflated 64%)
updating: results/checkpoint-75/training_args.bin (deflated 52%)
updating: results/checkpoint-25/ (stored 0%)
updating: results/checkpoint-25/model.safetensors (deflated 15%)
updating: results/checkpoint-25/config.json (deflated 54%)
updating: results/checkpoint-25/rng_state.pth (deflated 24%)
updating: results/checkpoint-25/optimizer.pt (deflated 28%)
updating: results/checkpoint-25/scheduler.pt (deflated 56%)
updating: results/checkpoint-25/trainer_state.json (deflated 57%)
updating: results/checkpoint-25/training_args.bin (deflated 52%)
updatin

In [47]:
# @title Step 6: Test Your Model
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)


Device set to use cpu


In [48]:
test_comments = [
    "Your support team resolved my issue in minutes!",
    "The product arrived damaged and poorly made",
    "Materials feel premium and durable",
    "Not worth the premium price tag",
    "Way overpriced for what you get",
    "Super fast shipping, impressive",
    "Tracking information was never updated"
]

for comment in test_comments:
    result = classifier(comment)
    label_idx = int(result[0]['label'].split('_')[-1])
    label = le.inverse_transform([label_idx])[0]
    print(f"\"{comment}\"")
    print(f"→ {label} ({result[0]['score']:.2%} confidence)")
    print("-" * 50)

"Your support team resolved my issue in minutes!"
→ service (66.19% confidence)
--------------------------------------------------
"The product arrived damaged and poorly made"
→ quality (71.50% confidence)
--------------------------------------------------
"Materials feel premium and durable"
→ quality (82.25% confidence)
--------------------------------------------------
"Not worth the premium price tag"
→ price (82.15% confidence)
--------------------------------------------------
"Way overpriced for what you get"
→ price (82.30% confidence)
--------------------------------------------------
"Super fast shipping, impressive"
→ delivery (48.33% confidence)
--------------------------------------------------
"Tracking information was never updated"
→ quality (34.43% confidence)
--------------------------------------------------
